In [2]:
import os
import cv2
from PIL import Image, ImageEnhance, ImageOps, ImageFilter
import random
import numpy as np
from matplotlib import pyplot as plt
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.io import read_image
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import torch.optim as optim
from torch.utils.data import DataLoader

In [3]:
# Define the directory containing the augmented images
"""
y'all should copy the directory where you have stored your files on your local PC
"""
data_dir = "C:\\Users\\wisea\\Transfer\\Deep_Learning_Files\\ProjectFilles\\Data"

# Define the transformations to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [4]:
# Function to load and augment images from a directory
def augment_images(directory, num_augments):
    augmented_images = []
    for filename in os.listdir(directory):
        # Load the original image
        image_path = os.path.join(directory, filename)
        original_image = Image.open(image_path)
        
        # Apply transformations to the original image
        for _ in range(num_augments):
            # Randomly select augmentation techniques
            chosen_transforms = random.sample(transform_options, k=random.randint(1, max_transforms))
            composed_transform = transforms.Compose(chosen_transforms)
            augmented_image = composed_transform(original_image)
            augmented_images.append(transform(augmented_image))
    return augmented_images

In [5]:
# Define the subfolders containing the images
subfolders = ['Alopecia-Pictures', 'seborrheic-dermatitis-pictures', 'Psoriasis-pictures']

# Create a dictionary to store augmented images for each subfolder
augmented_images_by_subfolder = {}

In [6]:
# Define augmentation strategy
augmentation_strategy = {
    'Alopecia-Pictures': {
        'less_than_50': 6,
        'between_50_and_100': 4,
        'more_than_100': 2
    },
    'seborrheic-dermatitis-pictures': {
        'less_than_50': 6,
        'between_50_and_100': 4,
        'more_than_100': 2
    },
    'Psoriasis-pictures': {
        'less_than_50': 6,
        'between_50_and_100': 4,
        'more_than_100': 2
    }
}

In [7]:
# Augmentation techniques
transform_options = [
    transforms.RandomRotation(degrees=30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5)
]
max_transforms = len(transform_options)

In [8]:
# Augment images for each subfolder based on the strategy
for subfolder in subfolders:
    subfolder_path = os.path.join(data_dir, subfolder)
    num_images = len(os.listdir(subfolder_path))
    num_augments = None
    if num_images < 50:
        num_augments = augmentation_strategy[subfolder]['less_than_50']
    elif 50 <= num_images <= 100:
        num_augments = augmentation_strategy[subfolder]['between_50_and_100']
    else:
        num_augments = augmentation_strategy[subfolder]['more_than_100']
        
    augmented_images = augment_images(subfolder_path, num_augments=num_augments)
    augmented_images_by_subfolder[subfolder] = augmented_images

In [30]:
# Convert the augmented images to torch tensors
augmented_images_tensors_by_subfolder = {}
for subfolder, images in augmented_images_by_subfolder.items():
    augmented_images_tensors_by_subfolder[subfolder] = torch.stack(images)

# Display the size of tensors for each subfolder
for subfolder, tensors in augmented_images_tensors_by_subfolder.items():
    print(f"Subfolder: {subfolder}, Tensor size: {tensors.size()}")

Subfolder: Alopecia-Pictures, Tensor size: torch.Size([328, 3, 224, 224])
Subfolder: seborrheic-dermatitis-pictures, Tensor size: torch.Size([240, 3, 224, 224])
Subfolder: Psoriasis-pictures, Tensor size: torch.Size([202, 3, 224, 224])


In [31]:
# Display 20 pictures from each subfolder
for subfolder, images in augmented_images_by_subfolder.items():
    print(f"\nShowing images from subfolder: {subfolder}")
    plt.figure(figsize=(20, 5))
    for i in range(20):  # Displaying 20 images from each subfolder
        plt.subplot(2, 10, i + 1)
        plt.imshow(images[i].permute(1, 2, 0))
        plt.axis('off')
    plt.show()


Showing images from subfolder: Alopecia-Pictures


In [11]:
# Combine all augmented images and create labels
all_images = torch.cat([torch.stack(images) for images in augmented_images_by_subfolder.values()], dim=0)
num_classes = len(subfolders)
all_labels = torch.tensor(sum([[i] * len(images) for i, images in enumerate(augmented_images_by_subfolder.values())], []))

In [12]:
# Split data into training and validation sets
dataset = list(zip(all_images, all_labels))
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [18]:
# Define DataLoader for training and validation sets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Check the sizes of training and validation sets
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

Training set size: 616
Validation set size: 154


In [19]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

In [20]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, cardinality=32):
        super(ResidualBlock, self).__init__()
        mid_channels = out_channels // 2
        self.conv1 = ConvBlock(in_channels, mid_channels, kernel_size=1)
        self.conv2 = ConvBlock(mid_channels, mid_channels, kernel_size=3, stride=stride, padding=1)
        self.conv3 = nn.Conv2d(mid_channels, out_channels, kernel_size=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        self.cardinality = cardinality

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out += self.downsample(residual)
        out = self.bn(out)
        out = self.relu(out)
        return out

In [21]:
class ResNeXt(nn.Module):
    def __init__(self, num_blocks, cardinality, num_classes=3):
        super(ResNeXt, self).__init__()
        self.conv1 = ConvBlock(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self.make_layer(64, 256, num_blocks[0], stride=1, cardinality=cardinality)
        self.layer2 = self.make_layer(256, 512, num_blocks[1], stride=2, cardinality=cardinality)
        self.layer3 = self.make_layer(512, 1024, num_blocks[2], stride=2, cardinality=cardinality)
        self.layer4 = self.make_layer(1024, 2048, num_blocks[3], stride=2, cardinality=cardinality)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, num_classes)

    def make_layer(self, in_channels, out_channels, num_blocks, stride, cardinality):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride, cardinality))
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels, cardinality=cardinality))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


In [23]:
# Instantiate the model
model = ResNeXt(num_blocks=[3, 4, 6, 3], cardinality=32, num_classes=3).to(device)
print(model)

ResNeXt(
  (conv1): ConvBlock(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): ConvBlock(
        (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (conv2): ConvBlock(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [24]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [25]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total

    # Validation loop
    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_labels)
            val_running_loss += val_loss.item()
            _, val_predicted = torch.max(val_outputs, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()
    val_loss = val_running_loss / len(val_loader)
    val_accuracy = val_correct / val_total

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

Epoch 1/10, Train Loss: 1.0055, Train Acc: 0.5503, Val Loss: 3.7804, Val Acc: 0.3247
Epoch 2/10, Train Loss: 0.7622, Train Acc: 0.6753, Val Loss: 1.0915, Val Acc: 0.5390
Epoch 3/10, Train Loss: 0.7787, Train Acc: 0.6429, Val Loss: 1.0122, Val Acc: 0.5390
Epoch 4/10, Train Loss: 0.7294, Train Acc: 0.6737, Val Loss: 0.8164, Val Acc: 0.6169
Epoch 5/10, Train Loss: 0.7765, Train Acc: 0.6591, Val Loss: 1.3404, Val Acc: 0.4416
Epoch 6/10, Train Loss: 0.7787, Train Acc: 0.6786, Val Loss: 1.3730, Val Acc: 0.4740
Epoch 7/10, Train Loss: 0.7713, Train Acc: 0.6981, Val Loss: 1.1110, Val Acc: 0.5779
Epoch 8/10, Train Loss: 0.7679, Train Acc: 0.6672, Val Loss: 1.3210, Val Acc: 0.3896
Epoch 9/10, Train Loss: 0.6718, Train Acc: 0.7240, Val Loss: 0.7368, Val Acc: 0.6948
Epoch 10/10, Train Loss: 0.6523, Train Acc: 0.7305, Val Loss: 2.8089, Val Acc: 0.3961


In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Inside the train_model function
for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)